In [ ]:
import os
import numpy as np
import easyocr
from tqdm.notebook import trange
from doc2text.reader import Reader
from metric.metric import cosine_similarity, cer

def_reader = easyocr.Reader(['ru'])
old_reader = easyocr.Reader(
            ['ru'],
            gpu=True,
            model_storage_directory='C:/shiftlab_easy_ocr/doc2text/weights/easyOCR/model',
            user_network_directory='C:/shiftlab_easy_ocr/doc2text/weights/easyOCR/user_network',
            download_enabled=False,
            recog_network='ru_custom'
        )
new_reader = Reader()

#get y_true
with open("./test/texts.txt", "r") as texts:
    y_true = texts.readlines()

#get apple
with open("./test/texts_apple.txt", "r") as texts_apple:
    apple = texts_apple.readlines() 

scores_def = []
scores_old = []
scores_new = []
scores_apple = []
cer_scores_def = []
cer_scores_old = []
cer_scores_new = []
cer_scores_apple = []
directory = 'test'
for k in trange(len(os.listdir(directory)) - 2):
    filename = str(k + 1) + ".jpg"
    print(filename)
    file_path = os.path.join(directory, filename)
    
    #model eval
    def_result = def_reader.readtext(file_path)
    def_str = ' '.join([word[1] for word in def_result]).lower()
    old_result = old_reader.readtext(file_path)
    old_str = ' '.join([word[1] for word in old_result]).lower()
    new_result = new_reader.doc2text(file_path)
    new_str = new_result[0].lower()
    print("default easyOCR:\n", def_str)
    print("trained easyOCR:\n", old_str)
    print("NEW model:\n", new_str)
    print("apple model:\n", apple[k].lower())
    print("y_true:\n", y_true[k])

    #metric
    scores_def.append(cosine_similarity(y_true[k], def_str))
    scores_old.append(cosine_similarity(y_true[k], old_str))
    scores_new.append(cosine_similarity(y_true[k], new_str))
    scores_apple.append(cosine_similarity(y_true[k], apple[k].lower()))
    cer_scores_def.append(cer(y_true[k], def_str))
    cer_scores_old.append(cer(y_true[k], old_str))
    cer_scores_new.append(cer(y_true[k], new_str))
    cer_scores_apple.append(cer(y_true[k], apple[k].lower()))

print("\n")
print("mean default EasyOCR cosine_similarity", np.mean(scores_def))        
print("mean our EasyOCR cosine_similarity", np.mean(scores_old))
print("mean NEW model cosine_similarity", np.mean(scores_new))
print("mean apple model cosine_similarity", np.mean(scores_apple))

print("mean default EasyOCR cer", np.mean(cer_scores_def))        
print("mean our EasyOCR cer", np.mean(cer_scores_old))
print("mean NEW model cer", np.mean(cer_scores_new))
print("mean apple model cer", np.mean(cer_scores_apple))

In [ ]:
print("default EasyOCR cosine_similarity", scores_def)
print("our EasyOCR cosine_similarity", scores_old)
print("NEW model cosine_similarity", scores_new)
print("apple model cosine_similarity", scores_apple)

print("mean default EasyOCR cer", cer_scores_def)
print("mean our EasyOCR cer", cer_scores_old)
print("mean NEW model cer", cer_scores_new)
print("mean apple model cer", cer_scores_apple)